In [301]:
#https://plotly.com/python/reference/

%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : "#1f1e1e"}

import io
import json
import os
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from IPython.display import Markdown, display, Image, clear_output
from PIL import Image as PILImage

# LOCAL GIT REPOSITORY LOCATION FOR https://github.com/CSSEGISandData/COVID-19
directory = '/Users/davidmorton/Documents/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'

!git -C $directory pull

states_to_drop = ['Puerto Rico', 'American Samoa', 'Guam', 'Northern Mariana Islands', 'Virgin Islands']

state_names = pd.read_csv('states.csv').set_index('State').to_dict()['Abbreviation']


remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 7 (delta 5), reused 7 (delta 5), pack-reused 0
Unpacking objects: 100% (7/7), done.
From https://github.com/CSSEGISandData/COVID-19
   cf381920..e0be60e4  web-data   -> origin/web-data
Already up to date.


In [334]:
def get_census_info():
    county_populations = pd.read_csv("co-est2019-alldata.csv", encoding = "ISO-8859-1")
    county_populations = county_populations[county_populations['SUMLEV'] == 50] # 40 is states, 50 is counties
    county_populations = county_populations[['STATE', 'COUNTY', 'CTYNAME', 'POPESTIMATE2019']] # Keep Just population estimate
    county_populations['fips'] = (county_populations['STATE'] * 1000) + county_populations['COUNTY']
    county_populations = county_populations[['fips', 'POPESTIMATE2019']]
    county_populations = county_populations.rename(columns={'POPESTIMATE2019':'pop'})
    county_populations = county_populations.set_index('fips')
    return county_populations

def load_csv(directory, f):
    result = pd.read_csv(directory + f)
    result['date'] = datetime.strptime(f, '%m-%d-%Y.csv')
    return result
    
def get_daily_report_data():
    result = pd.concat([load_csv(directory, f) for f in os.listdir(directory) if f.endswith(".csv")]).reset_index()
    result = result[~np.isnan(result['FIPS'])]
    result['FIPS'] = result['FIPS'].astype(int)
    result = result[~result['Province_State'].isin(['Diamond Princess', 'Grand Princess', 'Guam', 'Northern Mariana Islands','Puerto Rico', 'Virgin Islands', 'American Samoa'])]
    return result

def get_county_percentages():
    daily_report_data = get_daily_report_data()
    df1 = (
        daily_report_data
        [['Confirmed', 'FIPS', 'date']] # Keep these three columns
        .rename(columns={'Confirmed':'confirmed', 'FIPS':'fips'}) # Rename for simplicity
        .drop_duplicates(['date','fips']) # There are a few duplicates. Drop them. 
        .pivot(values='confirmed', index='date', columns='fips') # Create the pivot table. 
        .fillna(0)
        .diff()
        .rolling(window=7)
        .mean()
        .dropna()
        .T
    )

    df2 = get_census_info()
    return (df1.div(df2.iloc[:,0], axis=0).dropna()) * 100000

def get_allcounties():
    with open(os.getcwd() + '/geojson-counties-fips.json') as response:
        allcounties = json.load(response)
    return allcounties

def draw_choropleth(row, county_geojson, colorscale, zmax, store_intermediate_files):

    clear_output(wait=True)
    date = row.name
    print(date)
    
    foldername = 'gitignore-county-spread-images'
    filename = '{0}/{1:%Y-%m-%d-%H-%M}.png'.format(foldername, date)
    if store_intermediate_files == True:
        if (not os.path.exists(foldername)):
            os.mkdir(foldername)
        if (os.path.exists(filename)):
            return PILImage.open(filename)
    
    local_directory = os.getcwd()

    row = row[row.index.isin([stateid['id'] for stateid in county_geojson['features']])]

    row = row.reset_index().rename(columns={date:'confirmed'})

    for feature in county_geojson["features"]:
        feature["properties"]["FIPS"] = str(int(feature["properties"]["STATE"])) + feature["properties"]["COUNTY"]

    bgcolor = '#0d1238'
    fig = px.choropleth(row, 
                        geojson=county_geojson, 
                        locations='fips', 
                        color='confirmed',
                        color_continuous_scale=colorscale,
                        featureidkey='properties.FIPS',
                        projection='albers usa',
                        title="Where is COVID Spreading?",
                        template='plotly_dark',
                        range_color=[0,zmax],
                        labels={"confirmed": ""}
                      )

    fig.update_geos(
        showcountries=False,
        showcoastlines=True, 
        showframe=False,
        oceancolor=bgcolor,
        lakecolor=bgcolor,
        showland=False, 
        bgcolor=bgcolor)
   
    fig.update_layout(
        plot_bgcolor=bgcolor, 
        paper_bgcolor=bgcolor, 
        title_text = 'COVID-19 Rate of Spread',
        title_xanchor = 'center',
        title_x = 0.5,
        #tickvals=[0,20,40,80,100],
        #ticktext=["0","20","40","80","100+"],
        coloraxis=dict(
            colorbar=dict(
                #title="New Cases", 
                ticktext=["0","10","20","30", "40","50+"], 
                tickvals=[0,10,20,30,40,50]
            )
        ),
        annotations=[
            dict(
                x=-0.13,
                y=-0.23,
                font=dict(
                    color="#A1A1A1",
                    size=10
                ),
                showarrow=False,
                text='Data from Johns Hopkins University COVID-19 DataSet',
                xref="paper",
                yref="paper"
            ),
            dict(
                x=0.52,
                y=1.15,
                yanchor='top',
                xanchor='center',
                font=dict(
                    size=12
                ),
                showarrow=False,
                text='Weekly Rolling Average of New Cases Per 100,000 People',
                xref="paper",
                yref="paper"
            ),
            dict(
                x=0.52,
                y=0,
                yanchor='top',
                xanchor='center', 
                font=dict(size=12),
                showarrow=False,
                text = '{0:%B %d}'.format(date),
                xref='paper',
                yref='paper'
            )
        ]
    )
    #cbar = fig.colorbar()
    #print(list(fig.select_coloraxes())[0])
    fig.update_traces(marker_line_width=0)
    #fig.show()
    img_bytes = fig.to_image(format="png")
    image = PILImage.open(io.BytesIO(img_bytes))
    
    if store_intermediate_files == True:
        image.save(filename)
    return image

def draw_choropleth_image(row, county_geojson, colorscale, zmax):
    image = draw_choropleth(row, county_geojson, colorscale, zmax)
    url = 'covid19-county-cases.gif'
    image.save(url)
    return url + "?c=" + str(random.randint(0,2e9))

def draw_choropleth_for_animation(row, county_geojson, colorscale, zmax, i, datacount):
    update_progress(i/datacount)
    return draw_choropleth(row, county_geojson, colorscale, zmax)
    
def make_animation(totalduration, pausetime, timeinterpolation, theme, filename, store_intermediate_files):
    
    county_geojson=get_allcounties()
    originalData = get_county_percentages().T.clip(0, 50)
    data = originalData.resample(rule=timeinterpolation).asfreq().interpolate()
    
    adjduration = totalduration-pausetime;
    perslide = round((adjduration*1000)/len(data));
    repeatlast = round((pausetime*1000)/perslide)
    
    print('Frames will be shown for {0} milliseconds'.format(perslide))
    
    rows = [data.iloc[i] for i in range(0, len(data))]

    # Old binning method was too dark across the board. 
    #binned = np.unique(sorted(data.values.flatten().flatten()))
    binned = list(range(0,9))
    bins = pd.qcut(binned, q=8, retbins=True)[1]
    
    # Ended up being completely not what I expected...
    #binned = pd.Series((vals**vals.max())*(vals.max()**vals.max()*vals.max()/vals.max()**vals.max()))
    #bins = pd.qcut(binned, q=8, retbins=True)[1]
    
    #bins = ((((np.array(range(0,9))/8)**.3))-vals.min())*(vals.max()-vals.min())
    #vals = originalData.values.flatten().flatten()
    #colorstops = (((np.array(range(0,9))/8)**3))
    
    
    colorstops = (bins - bins.min()) / (bins.max() - bins.min())
    #print(colorstops)
    colorscale = list(zip(colorstops, getattr(px.colors.sequential, theme)))
    #print(colorscale)
    zmax = originalData.max().max()
    print(zmax)

    images = [draw_choropleth(row, county_geojson, colorscale, zmax, store_intermediate_files) for row in rows]
    images.extend([images[-1] for i in range(0, repeatlast)])
    
    url = filename
    images[0].save(url,
                   save_all=True, append_images=images[1:], optimize=True, duration=perslide, loop=0)
    clear_output()
    return url + "?c=" + str(random.randint(0,2e9))

In [331]:
# Quick test for visuals... Downsamples the values for speed, and doesn't save the intermediate pngs on the disk.
Image(url=make_animation(5, 1, '15D', 'Inferno', 'covid19-county-spread-90day.gif', False))

In [346]:
# Full Generation. Upsamples the data and stores the intermediate pngs on the disk. 
Image(url=make_animation(30, 5, '8H', 'Inferno', 'covid19-county-spread.gif', True))


In [ ]:
[os.system("printf '\a'") for i in range(0, 200)]; # BEEP!
